In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set_style('whitegrid')

import torch
from torch.autograd import Variable
from torch.utils import data
from torch import nn
from torch import optim
from torch.utils import data

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)


import sys
sys.path.append('..')

from sklearn.linear_model import RidgeCV, MultiTaskElasticNetCV

cuda:0


In [2]:
df_train = pd.read_csv( '../../data/processed/rf_train.csv')
df_test = pd.read_csv( '../../data/processed/rf_test.csv')

In [3]:
df_train.columns

Index(['dt', 'feature_0', 'feature_1', 'feature_2', 'feature_3', 'feature_4',
       'feature_5', 'feature_6', 'feature_7', 'feature_8', 'feature_9',
       'feature_10', 'feature_11', 'feature_12', 'feature_13', 'feature_14',
       'feature_15', 'feature_16', 'feature_17', 'feature_18', 'feature_19',
       'feature_20', 'feature_21', 'feature_22', 'feature_23', 'feature_24',
       'feature_25', 'feature_26', 'feature_27', 'feature_28', 'feature_29',
       'feature_30', 'feature_31', 'feature_32', 'feature_33', 'feature_34',
       'feature_35', 'feature_36', 'feature_37', 'y_0', 'y_1'],
      dtype='object')

In [4]:
num_dts = len(np.unique(df_train['dt']))
y_labels = ['y_0', 'y_1']
y_train = df_train.loc[df_train['dt'] == 0, y_labels].values
y_test = df_test.loc[df_train['dt'] == 0, y_labels].values

# First concatenate all timesteps together one weight per combination of time and feature
X_train = df_train.drop(labels=y_labels + ['dt'], axis=1).values.reshape(df_train.shape[0]//num_dts, -1)
X_test = df_test.drop(labels=y_labels + ['dt'], axis=1).values.reshape(df_test.shape[0]//num_dts, -1)

In [5]:
linear = RidgeCV(alphas=np.logspace(-10, 5, num=32))
linear.fit(X_train, y_train)

RidgeCV(alphas=array([1.00000e-10, 3.04699e-10, 9.28415e-10, 2.82887e-09, 8.61954e-09,
       2.62636e-08, 8.00250e-08, 2.43835e-07, 7.42964e-07, 2.26380e-06,
       6.89779e-06, 2.10175e-05, 6.40400e-05, 1.95129e-04, 5.94557e-04,
       1.81161e-03, 5.51995e-03, 1.68192e-02, 5.12481e-02, 1.56152e-01,
       4.75794e-01, 1.44974e+00, 4.41734e+00, 1.34596e+01, 4.10113e+01,
       1.24961e+02, 3.80755e+02, 1.16016e+03, 3.53498e+03, 1.07711e+04,
       3.28193e+04, 1.00000e+05]),
    cv=None, fit_intercept=True, gcv_mode=None, normalize=False,
    scoring=None, store_cv_values=False)

In [6]:
linear.score(X_train, y_train), linear.score(X_test, y_test)

(0.183042380132744, 0.10844945530163266)

In [7]:
y_test_hat = linear.predict(X_test)
y_test_hat

array([[ 1.11252544,  0.11454252],
       [ 1.34859902, -0.19781908],
       [ 1.42869322, -0.3084776 ],
       ...,
       [ 1.65033971,  0.2982384 ],
       [ 1.60091703,  0.42215438],
       [ 1.11190763,  0.73622077]])

In [8]:
y_labels = ['y_0', 'y_1']
y_train_np = df_train.loc[df_train['dt'] == 0, y_labels].values
y_test_np = df_test.loc[df_train['dt'] == 0, y_labels].values

num_dts = len(np.unique(df_train['dt']))
num_features = df_train.shape[1] - 3# # minus rows dt, y_0, y_1

X_train_np = df_train.drop(labels=y_labels + ['dt'], axis=1).values
X_test_np = df_test.drop(labels=y_labels + ['dt'], axis=1).values

# Reshape from (dts*kicks, features) to (features, dts, kicks)
X_train_np = X_train_np.reshape(X_train_np.shape[0]//num_dts, num_dts,num_features)
X_test_np = X_test_np.reshape(X_test_np.shape[0]//num_dts, num_dts, num_features)

# Transpose to (dts, kicks, features)
X_train_np = X_train_np.transpose(1,0, 2)
X_test_np = X_test_np.transpose(1,0, 2)

# To Tensors
X_train = torch.from_numpy(X_train_np).float()
y_train = torch.from_numpy(y_train_np).float()

X_test = torch.from_numpy(X_test_np).float()
y_test = torch.from_numpy(y_test_np).float()

X_train.shape

torch.Size([3, 20786, 38])

In [9]:
class LinearRF(nn.Module):
    def __init__(self, num_features, num_dts):
        super().__init__()
        # Treat each timestep as independent feature!
        out_size = 2
        self.num_features = num_features
        self.dt_weights = nn.Parameter(data=torch.ones(num_dts)/num_dts)#torch.rand(num_dts)/num_dts)
        print(self.dt_weights)
        
        self.linear = nn.Linear(in_features=self.num_features,
                                out_features=out_size)
        
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.orthogonal_(m.weight.data)
                m.bias.data.fill_(0.0)
        
    def forward(self, x):
        # Normalize dt_weights s.t. they are >= 0 and sum to 1
        dt_weights = nn.functional.softmax(self.dt_weights, dim=0)
    
        # Shape: examples, num_dts, output (2)
        x =  self.linear(x)

        x = x.permute(1,2,0)
        return (x * dt_weights).sum(dim=-1)
    
model = LinearRF(num_features=num_features, num_dts = num_dts).to(device)

Parameter containing:
tensor([ 0.3333,  0.3333,  0.3333])


In [10]:
best_loss = torch.tensor(float('inf')).to(device)
optimizer = optim.LBFGS(model.parameters())

class L2MSELoss(nn.Module):
    def __init__(self, lambda_reg, model):
        super().__init__()
        self.lambda_reg = lambda_reg
        self.mse = nn.MSELoss()
        self.model = model
        
    def forward(self, a, b):
        loss = self.mse(a, b)
        
        loss_reg = 0.0
        num_params = 0.0
        for m in self.model.modules():
            if isinstance(m, nn.Linear):
                loss_reg += (m.weight**2).sum()
                size = torch.tensor(m.weight.size()).prod()
                num_params += size
              
        #loss_reg += (self.model.dt_weights**2).sum()
        #num_params += self.model.dt_weights.shape[0]
        
        loss_reg = loss_reg.sqrt()/num_params.item()
        return loss + self.lambda_reg * loss_reg
        
criterion = L2MSELoss(lambda_reg=0.1, model=model).to(device)

while True:
    def eval_model():
        optimizer.zero_grad()
        out = model(X_train.to(device))
        loss = criterion(out, y_train.to(device))
        loss.backward()
        return loss
    cur_loss = optimizer.step(closure=eval_model)
    if best_loss - cur_loss > 10e-9:
        best_loss = cur_loss
        print(best_loss)
    else:
        break
    

tensor(1.4099, device='cuda:0')
tensor(0.4607, device='cuda:0')
tensor(0.4605, device='cuda:0')
tensor(0.4605, device='cuda:0')
0.4605485498905182


In [11]:
from sklearn.metrics import r2_score
y_train_hat = model(X_train.to(device)).cpu().data.numpy()
y_test_hat = model(X_test.to(device)).cpu().data.numpy()

r2_score(y_train_np, y_train_hat), r2_score(y_test_np, y_test_hat), nn.functional.softmax(model.dt_weights, dim=0)

(0.16727211640464051,
 0.10325354725643177,
 tensor([ 0.9015,  0.0695,  0.0290], device='cuda:0'))

In [12]:
model.dt_weights

Parameter containing:
tensor([ 2.3328, -0.2299, -1.1029], device='cuda:0')

In [13]:
from scipy.stats import spearmanr
print(spearmanr(y_train_np[:,0], y_train_hat[:,0]),'\n', 
      spearmanr(y_train_np[:,1], y_train_hat[:,1]),'\n\n',
      spearmanr(y_test_np[:,0], y_test_hat[:,0]),'\n', 
      spearmanr(y_test_np[:,1], y_test_hat[:,1]))      

SpearmanrResult(correlation=0.4243535038150776, pvalue=0.0) 
 SpearmanrResult(correlation=0.4672948414083257, pvalue=0.0) 

 SpearmanrResult(correlation=0.3273209706180686, pvalue=1.8279989868749893e-117) 
 SpearmanrResult(correlation=0.3903689916786351, pvalue=1.8894353695115036e-170)


In [14]:
y_train_hat.shape

(20786, 2)